In [86]:
import numpy as np;
import pandas as pd;
A = np.matrix("1 2; 3 4");
print(A);
# reader = open("D:/datasets/titanic-kaggle/train.csv", "r");
trainingDf= pd.read_csv("D:/datasets/titanic-kaggle/train.csv", index_col=0, parse_dates=True, nrows = 10);
# print(trainingDf);
trainingSet = np.loadtxt(open("D://datasets//titanic-kaggle//gender_submission.csv", "r"), delimiter=",", skiprows=1)
# print(trainingSet)
# trainingDf.pivot_table(values='PassengerId', index='PassengerId', columns='Sex', aggfunc=lambda x: len(x.unique()));
# print(pd.pivot_table(data=trainingDf, rows=["PassengerId"], cols=["Sex"], values=["PassengerId"], aggfunc='count'))
# addColumns = trainingDf.Sex.str.get_dummies();
# print(type(addColumns))
# trainingDf["Cabin"]=len(trainingDf["Cabin"])
listRowstoColumns=["Sex", "Embarked"];
listRowstoColumns=[Cabin];
trainingDf = changeCategorical(trainingDf, listRowstoColumns, listRowsToBoolean);

def changeCategorical(df, listRowstoColumns, listRowsToBoolean):
    return pd.concat(generateBooleanColumns(df, listRowsToBoolean) , generateValueColumns(df, listRowstoColumns));

def generateBooleanColumns(df, columns):
    for column in columns:
            df[column] = -1 * df[column].isnull() + 1
    return

def generateValueColumns(df, columns);
    finalDf = pd.DataFrame()
    for column in columns:
        pd.concat([finalDf,trainingDf],axis =1)
        
# trainingDf  = pd.concat([trainingDf.drop('Sex', axis=1), trainingDf['Sex'].str.get_dummies()])
print(trainingDf['Sex'].str.get_dummies())
# print(pd.concat([trainingSet, addColumns],axis=1))

def 

SyntaxError: invalid syntax (<ipython-input-86-91cdd7fa34f7>, line 27)

In [89]:
trainingDf["new"] = -1 * trainingDf.Cabin.isnull() + 1
# trainingDf
print(pd.concat([pd.DataFrame(),trainingDf],axis =1))

             Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   
6                   0       3   
7                   0       1   
8                   0       3   
9                   1       3   
10                  1       2   

                                                          Name     Sex   Age  \
PassengerId                                                                    
1                                      Braund, Mr. Owen Harris    male  22.0   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
3                                       Heikkinen, Miss. Laina  female  26.0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
5                                     Allen, Mr. William Henry    male  35.0   
6                                          